# 📊 AWS Glue & PySpark ETL: Análise de Transações Financeiras

# 📌 1️⃣ Introdução e Objetivo

Este notebook implementa a etapa de **ETL (Extração, Transformação e Carga)** para processamento dos dados brutos.

📌 **Objetivo:**  
- Carregar os dados brutos da pasta `data/raw/`
- Aplicar transformações com **PySpark** (limpeza, normalização e regras de negócio)
- Armazenar os dados processados em formato **Parquet** para análise posterior

🔧 **Tecnologias utilizadas:**  
- **PySpark** para processamento eficiente de grandes volumes de dados
- **AWS Glue** (se rodarmos na nuvem) ou Spark local
- **Parquet** como formato de armazenamento otimizado

---


## 📌 2️⃣ Configuração do do PySpark

In [ ]:
import os
import pyspark
from pyspark.sql import SparkSession

# Criar uma sessão do PySpark
spark = SparkSession.builder \
    .appName("ETL_Processing") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

print("✅ PySpark configurado com sucesso!")

## 📌 3️⃣ Carregamento dos Dados Brutos

In [ ]:
import os
import yaml

# Carregar Configuração do YAML
config_path = os.path.abspath(os.path.join(os.getcwd(), "..", "config/config.yaml"))  

print(f"Tentando carregar: {config_path}")

if os.path.exists(config_path):
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    print("✅ Configuração carregada com sucesso!")
else:
    raise FileNotFoundError("❌ Arquivo 'config.yaml' não encontrado!")

# Ajustar caminhos do config.yaml para evitar problemas com barras no Windows/Linux
raw_data_path = os.path.normpath(config.get("raw_data_path", "data/raw/"))

# Garantir que o caminho seja absoluto
if not os.path.isabs(raw_data_path):
    raw_data_path = os.path.abspath(raw_data_path)

# Verificar se o diretório existe
if not os.path.exists(raw_data_path):
    raise FileNotFoundError(f"❌ ERRO: O diretório de dados brutos '{raw_data_path}' não existe!")

print(f"📂 Diretório de dados brutos encontrado: {raw_data_path}")

# 📌 4️⃣ Carregamento dos Dados Brutos

# Listar arquivos na pasta raw
arquivos_raw = os.listdir(raw_data_path)
print(f"📂 Arquivos encontrados: {arquivos_raw}")

# Verificar se há pelo menos um arquivo CSV na pasta
csv_files = [f for f in arquivos_raw if f.endswith(".csv")]
if not csv_files:
    raise FileNotFoundError("❌ Nenhum arquivo CSV encontrado na pasta de dados brutos.")

# Carregar os dados no PySpark DataFrame
df_raw = spark.read.csv(os.path.join(raw_data_path, csv_files[0]), header=True, inferSchema=True)

# Exibir amostra dos dados
print("✅ Exibindo amostra dos dados:")
df_raw.show(5)

# Exibir schema dos dados
print("✅ Schema do DataFrame:")
df_raw.printSchema()
